In [ ]:
print('Inicializando Variáveis do Programa')
#inicializando variáveis
#lista_confianças = [0.01]
lista_confianças = [0.01, 0.05]
variável_dependente = 'qty_2019'
variáveis_independentes = ('qty_2014', 'porcent_pagos')
ano_início = 2014
ano_fim = 2020
arquivo_indicadores_doutorado = "./indicadores_de_artigos_publicados.pkl"
max_variáveis_independentes_para_combinação = 5

lista_indicadores_a_ignorar = ['id', 
                                'index',
                                'pagtos',
                                'capes',
                                'indicador_tipo',
                                'indicador_grupo',
                                'indicador',
                                'qty_2012',
                                'qty_2013',
                                'qty_2014',
                                'qty_2015',
                                'qty_2016',
                                'qty_2017',
                                'qty_2018',
                                'qty_2019',
                                'tipo',
                                'nome',
                                'grupo',
                                'path',
                                'const',
                                'bool_cnpq',
                                'bool_capes',
                              ]


In [ ]:
from datetime import datetime
print('Início em: ', datetime.now())

print('Importações Iniciais')

from tkinter import NONE
from Lattes import Lattes
from Carga import Carga
from Database import Database
import psycopg2, pickle
import pandas, os
import statsmodels.api as sm
from urllib.parse import urlencode
from psycopg2.extensions import AsIs
from Carga import Carga
from pandas import pandas as pd
import numpy as np
import Database
import json, math
import locale
import statsmodels.formula.api as sm2
from itertools import combinations
from unidecode import unidecode



print('Fim: ', datetime.now())

In [ ]:
sql = '''
select * from financiamentos_doutorandos_2014
    inner join indicadores_doutorado
        on indicadores_doutorado.id = financiamentos_doutorandos_2014.id
    inner join indicadores_nomes
        on indicadores_nomes.tipo = indicadores_doutorado.indicador_tipo                 
'''

In [ ]:
#Carregar arquivo na memória
print('Carregando indicadores na memória. Início em: ', datetime.now())

db = Database.Database('CNPq')
engine = Carga.db_engine()

dt = pd.read_sql(sql, engine)
print(f'dt.size: {dt.size}')

print('Término em: ', datetime.now())

In [ ]:
sql_ids_pareados = '''
select pareado_1 as id from public.par_doutorandos_2014_sem_reposicao
UNION select pareado_2 as id from public.par_doutorandos_2014_sem_reposicao
UNION select pareado_3 as id from public.par_doutorandos_2014_sem_reposicao
'''

In [ ]:
print('Carregando indicadores dos pareados na memória. Início em: ', datetime.now())

db = Database.Database('CNPq')
engine = Carga.db_engine()

pareados = pd.read_sql(sql_ids_pareados, engine)
print(f'dt.size: {dt.size}')

print('Término em: ', datetime.now())

In [ ]:
## Normalizando as Tabelas

print('Realizando normalização da tabela. Início em: ', datetime.now())

dt['const'] = 1
dt = dt.loc[:,~dt.columns.duplicated()]

dt['bool_cnpq'] = dt['pagtos'].map(lambda pagtos: 1 if pagtos > 0 else 0)
dt['bool_capes'] = dt['capes'].map(lambda capes: 1 if capes == True else 0)


#Preenchendo Nan com zeros nos indicadores
dt.qty_2012 = dt.qty_2012.fillna(0)
dt.qty_2013 = dt.qty_2013.fillna(0)
dt.qty_2014 = dt.qty_2014.fillna(0)
dt.qty_2015 = dt.qty_2015.fillna(0)
dt.qty_2016 = dt.qty_2016.fillna(0)
dt.qty_2017 = dt.qty_2017.fillna(0)
dt.qty_2018 = dt.qty_2018.fillna(0)
dt.qty_2019 = dt.qty_2019.fillna(0)
dt.capes = dt.capes.fillna(0)
dt.pagtos = dt.pagtos.fillna(0)

valor_total_cnpq =  dt[['id', 'pagtos']].loc[dt['pagtos']>0].drop_duplicates().sum().pagtos
dt['porcent_pagos'] = dt['pagtos']/valor_total_cnpq


print('Fim em: ', datetime.now())


In [ ]:
#Apagando os financiados pela CAPES
dt.drop(dt.loc[dt.bool_capes==True].index, inplace=True)

#Apagando os que não foram escolhidos pelo pareamento
dt.drop(dt.loc[~dt.id.isin(pareados.id)].loc[dt.pagtos==0].index, inplace=True)

In [ ]:
def regressão(X,y):
    try:
        # OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
        reg = sm.OLS(y, X).fit()
        # mostrando as estatísticas do modelo
        #reg.summary()
        
        
        
        #guardando a regressão na memória

        result = {
                'Indicador': indicador,
                'Erro': False,
                'Prob (F-statistic)':reg.f_pvalue, 
                'Parâmetros': reg.params, 
                "P>|t|":reg.pvalues, 
                'Standard Error': reg.bse, 
                "Covariância entre os pasrâmetros": reg.normalized_cov_params, 
                'Number of observations n.': reg.nobs, 
                "R2 ajustado": reg.rsquared_adj, 
                "Modelo":reg.model,
            }
    except:
        result = {
                'Indicador': indicador,
                'Erro': True,
                'Prob (F-statistic)':None, 
                'Parâmetros': None, 
                "P>|t|":None, 
                'Standard Error': None, 
                "Covariância entre os pasrâmetros": None, 
                'Number of observations n.': None, 
                "R2 ajustado": None, 
                "Modelo": None,
            }
    return result

In [ ]:
def regressão_r2(dados, variável_dependente, variáveis_independentes, menor_R2 = True, max_variáveis_independentes_para_combinação = 5):
    
    if not 'indicador' in locals(): indicador = ''
    lista_formulas = []
    num_combinações = 0
    if len(variáveis_independentes) < max_variáveis_independentes_para_combinação:
        max_variáveis_independentes_para_combinação = len(variáveis_independentes)
     
    if menor_R2:
        for x in range(1,max_variáveis_independentes_para_combinação):
            for a in combinations(variáveis_independentes,x):
                if not a[0] == '-1':
                    lista_formulas.append(faz_fórmula(variável_dependente, a))
                    num_combinações += 1
    else:
        lista_formulas = list(faz_fórmula(variável_dependente, variáveis_independentes))
        num_combinações += 1
    
    list_results = []
    curr = 0
    for formula in lista_formulas:
        curr += 1
        print(f'{curr}/{num_combinações}: {curr*100/num_combinações}%.')
    
        try:
            # OLS vem de Ordinary Least Squares e o método fit irá treinar o modelo
            reg_ajustado = sm2.ols(formula, data = dados)
            reg = reg_ajustado.fit()
            # mostrando as estatísticas do modelo
            print('\nCom constante:\n',reg.summary2())  

            #guardando a regressão na memória
            

            result = {
                    'Indicador': indicador,
                    'Erro': False,
                    'Prob (F-statistic)':reg.f_pvalue, 
                    'Parâmetros': reg.params, 
                    "P>|t|":reg.pvalues, 
                    'Standard Error': reg.bse, 
                    "Covariância entre os pasrâmetros": reg.normalized_cov_params, 
                    'Number of observations n.': reg.nobs, 
                    "R2 ajustado": 0 if reg.rsquared_adj is None else reg.rsquared_adj,
                    "Modelo":reg.model,
                }

        except:
            result = {
                    'Indicador': indicador,
                    'Erro': True,
                    'Prob (F-statistic)':None, 
                    'Parâmetros': None, 
                    "P>|t|":None, 
                    'Standard Error': None, 
                    "Covariância entre os pasrâmetros": None, 
                    'Number of observations n.': None, 
                    "R2 ajustado": 0, 
                    "Modelo": None,
                }
    
        list_results.append(result)
        
    print(list_results)
    if menor_R2:
        maiorR = max(list_results, key=lambda x:x['R2 ajustado'])
    else:
        maiorR = list_results[0]
        
        
    return maiorR, list_results

In [ ]:
def regressão_com_confiança_2 (dados, 
                             variável_dependente, 
                             variáveis_independentes, 
                             confiança = 0.05,
                              menor_R2 = True,
                              max_variáveis_independentes_para_combinação = 5): 
    result, list_results = regressão_r2 (dados, 
                              variável_dependente, 
                              variáveis_independentes,
                              menor_R2, 
                          max_variáveis_independentes_para_combinação)

    while result['Erro'] == False and result['P>|t|'].max() > confiança:
        X = X.drop(columns=result['P>|t|'].idxmax())
        result = regressão(X,y)
        #print(result)
        
        
    if result['Erro'] == False and result['Prob (F-statistic)'] < confiança and result['P>|t|'].max() < confiança:
        result['Confiança'] = confiança
        return result
    else:
        result['Confiança'] = 'Erro'
    return result


In [ ]:
# Para teste sobre se outros indicadores são significativoas no indicador 'Artigo Publicado'
#Cria tabela com os ids e cada indicador do ano de 2014, colocando 0 para os ausentes.
#O indicador de 2019 que permanece é o de 'Artigo Publicado'
#Assim, regressão pode ser realizada para saber 
#   quais indicadores se correlacionam com o estado de 'Artigo Publicado' no ano de 2019.

list_names = dt.nome.unique()
if os.path.isfile(arquivo_indicadores_doutorado):
    indicadores = pd.read_pickle(arquivo_indicadores_doutorado)  
else:      
    
    list_ind = []
    unique_ids = dt.id.unique()
    total = len(unique_ids)
    curr = 0
    for id in unique_ids:
        #print(id)
        id_ind = dt.loc[dt.id==id]
        temp={
            'id': id,
            'porcent_pagos': id_ind.porcent_pagos.iloc[0],}
        try:
            temp['qty_2019'] = id_ind.loc[id_ind.nome == 'Artigo Publicado'].qty_2019.iloc[0]
        except:
            temp['qty_2019'] = 0

        for nome in list_names:
            if nome not in lista_indicadores_a_ignorar:
                try:
                    temp[nome] = id_ind.loc[dt.nome == nome].qty_2014.iloc[0]
                except:
                    temp[nome] = 0
        list_ind.append(temp)
        curr += 1
        if curr%10 == 0:
            print (f'{curr}/{total}: {curr*100/total}% done.')
    indicadores = pd.DataFrame(list_ind)
    del list_ind
    columns = {}
    for column in indicadores:
        new_column_name = unidecode(column.lower().replace(' ', '_').replace(',', '_').replace('/', '_').replace('-', '_'))
        lista_colunas_nomes_para_alterar = new_column_name
        columns[column] = lista_colunas_nomes_para_alterar
    indicadores.rename(columns=columns, inplace=True)
    indicadores.to_pickle(arquivo_indicadores_doutorado) 
list_variáveis_independentes = list(variáveis_independentes)
for column in indicadores:
    list_variáveis_independentes.extend([column])
 

In [ ]:
def faz_fórmula(variável_dependente, a):
    
    formula = f'{variável_dependente} ~ '
    for dado in a:
        if dado not in (lista_indicadores_a_ignorar):
            formula = formula + f'{dado} + '
    formula = formula[:-3]    
    
    return formula

In [ ]:
p_max_value = 1
for ind in lista_indicadores_a_ignorar:
    try:
        list_variáveis_independentes.remove(ind)
        print(f'\nEliminando a chave {ind}, que está na lista de eliminação.') 
    except:
        pass
while p_max_value > 0.01 and len (list_variáveis_independentes) > 0:
    formula = faz_fórmula(variável_dependente, list_variáveis_independentes)

    reg_ajustado = sm2.ols(formula, data = indicadores)
    reg = reg_ajustado.fit()
    # mostrando as estatísticas do modelo
    
    max_p = reg.pvalues.nlargest(1)
    p_max_value = max_p.values[0]
    p_max_key = max_p.keys()[0]
    if p_max_value < 0.01: 
        print(f'\nNenhuma chave a eliminar. Terminando. Maior P|t| = {p_max_value} é a chave {p_max_key}.')
        break
    
    print(f'\nEliminando a chave {p_max_key} com P|t| = {p_max_value}.') 
    list_variáveis_independentes.remove(p_max_key)   
    
    for k, v in reg.pvalues.iteritems():
        if math.isnan(v):
            print(f'\nEliminando a chave {k} com P|t| = {v}.') 
            list_variáveis_independentes.remove(k)   

len(list_variáveis_independentes)

In [ ]:
reg.summary2()

In [ ]:
result, list_results = regressão_r2(indicadores, variável_dependente, list_variáveis_independentes, menor_R2 = True, max_variáveis_independentes_para_combinação = 4)

In [ ]:
result

In [ ]:
reg_ajustado = sm2.ols('qty_2019 ~ artigo_publicado + porcent_pagos ', data = indicadores)
reg = reg_ajustado.fit()
# mostrando as estatísticas do modelo
print(reg.summary2())  


In [ ]:
#inicializando variáveis
lista_regressões = []
tempo_início = datetime.now()
feitos = 0
print(f'Iniciado em {tempo_início}.')

#pegando lista de todos os indicadores
todos_indicadores = dt.nome.unique()

#iterando sobre cada indicador
número_total = len(todos_indicadores)
for indicador in todos_indicadores:
    
    #Realizando previsão de término
    if not feitos == 0:
        porcentagem_já_feita = (feitos/número_total)
        tempo_passado = datetime.now() - tempo_início
        tempo_por_id = tempo_passado / feitos
        tempo_restante = (número_total - feitos) * tempo_por_id
        tempo_em_que_vai_acabar = datetime.now() + tempo_restante
        print(f'{feitos}/{número_total}. {porcentagem_já_feita * 100}% feitos. Fazendo indicador: {indicador}. Acabará em {tempo_em_que_vai_acabar}. O Último demorou: {datetime.now() - tempo_último}.')
    else:
        print(f'{feitos}/{número_total}. Fazendo id: {indicador}.')
    feitos += 1
    tempo_último = datetime.now()

    #Realizando a regressão
    dados = dt.loc[dt.nome==indicador]
    #X = dados[['const','qty_2014', 'bool_cnpq']]
    #y = dados[['qty_2019']]
    #lista_confianças = (0.001, 0.01, 0.05, 0.1)
    result, list_results = regressão_r2(dados, variável_dependente, variáveis_independentes, max_variáveis_independentes_para_combinação)


  

    lista_regressões.append(result)


In [ ]:
regressões = pd.DataFrame(lista_regressões)
regressões = pd.concat([regressões, 
                        regressões['Parâmetros'].apply(pd.Series), 
                        regressões['P>|t|'].apply(pd.Series), 
                        regressões['Standard Error'].apply(pd.Series)], 
                       axis=1)
regressões.to_excel(f'Regressões dos Doutorandos {ano_início} com Pareamento e sem Reposição.xlsx')

def real_br_money_mask(my_value):
    a = '{:,.2f}'.format(float(my_value))
    b = a.replace(',','v')
    c = b.replace('.',',')
    return c.replace('v','.')

Impacto = []
for indicador in dt.indicador_tipo.unique():
    
    try:
        delta_cnpq = regressões.loc[regressões.Indicador == indicador].Parâmetros.iloc[0].loc['percentual_pago']
        tamanho_amastra = len(dt[['id', 'pagtos']].loc[dt['pagtos']>0].loc[dt['indicador_tipo'] == indicador].drop_duplicates())
        #efetividade = tamanho_amastra * delta_cnpq
        efetividade = delta_cnpq
        
        temp = {
            'Indicador': indicador,
            'Tamanho Amostra:': tamanho_amastra,
            'Delta (CNPq): ': delta_cnpq,
            'Efetividade: ': efetividade,
            'Custo: ': real_br_money_mask(valor_total_cnpq),
            'Custo-efetividade: ': real_br_money_mask(valor_total_cnpq/efetividade),
        }
        Impacto.append(temp)
        print(temp)
    except:
        pass

impacto = pd.DataFrame(Impacto)
impacto.to_excel(f'Impacto dos Doutorandos {ano_início} com Pareamento e Não Reposição - confiança 001.xlsx')

In [ ]:
modelo_ajustado = sm2.ols(formula = 'qty_2019 ~ qty_2014 + C(bool_cnpq)', data = df)
modelo_treinado = modelo_ajustado.fit()
modelo_treinado.summary()

In [ ]:
len(df.loc[df.bool_cnpq==True])*modelo_treinado.params[1]

In [ ]:
df.loc[df.qty_2014 == 0].loc[df.pagtos==0].qty_2014.sum()

In [ ]:
(df.loc[df.qty_2014 == 0].loc[df.pagtos==0].qty_2019.sum())/len(df.loc[df.qty_2014 == 0].loc[df.pagtos==0].qty_2014)

In [ ]:
modelo_ajustado = sm2.ols(formula = 'qty_2019 ~ qty_2014 + pagtos -1', data = df)
modelo_treinado = modelo_ajustado.fit()
modelo_treinado.summary2()

In [ ]:
valor_total_cnpq * modelo_treinado.params.pagtos   

In [ ]:
1/modelo_treinado.params.pagtos

In [ ]:
modelo_treinado.rsquared_adj

In [ ]:
df_cnpq = df.loc[df.pagtos > 0]
df_par = df.loc[df.pagtos == 0]

In [ ]:
df_cnpq.qty_2014.sum()

In [ ]:
df_cnpq.qty_2019.sum()

In [ ]:
df_par.qty_2014.sum()

In [ ]:
df_par.qty_2019.sum()

In [ ]:
df_par.qty_2019.sum()/df_par.qty_2014.sum()

In [ ]:
df_cnpq.qty_2019.sum()/df_cnpq.qty_2014.sum()

In [ ]:
((df_cnpq.qty_2019.sum()/df_cnpq.qty_2014.sum())-(df_par.qty_2019.sum()/df_par.qty_2014.sum()))*df_cnpq.qty_2014.sum()